## Modul Macro - Copy Rows

    Subrutina CopyMatchingProjectsNoDuplicates():
        Creează o macrocomandă pentru copierea rândurilor corespunzătoare dintr-o foaie de calcul sursă într-o foaie țintă, bazat pe coduri de proiect specificate într-o a treia foaie, evitând duplicarea rândurilor deja copiate.
    Definirea variabilelor:
        sourceSheet, targetSheet, checkSheet: Foile de calcul pentru sursă, țintă și verificare, respectiv.
        sourceLastRow, checkLastRow, targetLastRow: Ultimul rând cu date din fiecare foaie respectivă.
        sourceCell, checkCell, targetCell: Celulele curente în procesul de iterație prin foile respective.
        isCopied: Un indicator boolean pentru a verifica dacă un rând a fost deja copiat în foaia țintă.
    Configurarea foilor:
        Foile sunt setate explicit la începutul subrutinei prin numirea lor (Sheet1, Sheet2, Sheet3).
    Determinarea ultimului rând cu date:
        Calculează ultimul rând cu date din fiecare foaie folosind coloana "A".
    Procesul de iterație:
        Începe de la rândul 2 pentru foaia de verificare (checkSheet) pentru a păstra antetul tabelului neatins.
        Verifică dacă codul de proiect din checkSheet este deja copiat în targetSheet.
        Dacă codul de proiect nu este găsit în targetSheet, caută rândul corespunzător în sourceSheet și îl copiază în targetSheet.
        Actualizează rândul țintă după fiecare copiere pentru a evita suprascrierea.
    Logica de evitare a duplicatelor:
        Verifică fiecare cod de proiect din foaia de verificare (checkSheet) împotriva codurilor de proiect din foaia țintă (targetSheet) pentru a preveni duplicarea rândurilor copiate.
        Folosește variabila isCopied pentru a marca dacă un rând a fost deja copiat, evitând astfel copierea multiplă a aceluiași rând.
    Copierea condiționată a rândurilor:
        Rândurile din sourceSheet sunt copiate în targetSheet doar dacă codul de proiect corespunde și nu a fost deja copiat, asigurând o copiere unică pentru fiecare proiect verificat.

In [ ]:
Sub CopyMatchingProjectsNoDuplicates()
    Dim sourceSheet As Worksheet, targetSheet As Worksheet, checkSheet As Worksheet
    Dim sourceLastRow As Long, checkLastRow As Long, targetLastRow As Long
    Dim sourceCell As Range, checkCell As Range, targetCell As Range
    Dim isCopied As Boolean

    ' Define the worksheets
    Set sourceSheet = ThisWorkbook.Sheets("Sheet1")
    Set checkSheet = ThisWorkbook.Sheets("Sheet2")
    Set targetSheet = ThisWorkbook.Sheets("Sheet3")

    ' Find the last row of data in each sheet
    sourceLastRow = sourceSheet.Cells(sourceSheet.Rows.Count, "A").End(xlUp).Row
    checkLastRow = checkSheet.Cells(checkSheet.Rows.Count, "A").End(xlUp).Row
    targetLastRow = targetSheet.Cells(targetSheet.Rows.Count, "A").End(xlUp).Row + 1

    ' Loop from row 2 in CheckSheet
    For Each checkCell In checkSheet.Range("A2:A" & checkLastRow)
        isCopied = False
        ' Check if the project code is already copied to TargetSheet
        For Each targetCell In targetSheet.Range("A2:A" & targetLastRow)
            If targetCell.Value = checkCell.Value Then
                isCopied = True
                Exit For
            End If
        Next targetCell

        ' If not copied, copy it from SourceSheet to TargetSheet
        If Not isCopied Then
            For Each sourceCell In sourceSheet.Range("A2:A" & sourceLastRow)
                If sourceCell.Value = checkCell.Value Then
                    ' Ensure copying starts at the correct targetLastRow
                    sourceSheet.Rows(sourceCell.Row).Copy Destination:=targetSheet.Rows(targetLastRow)
                    targetLastRow = targetLastRow + 1
                    Exit For ' Ensure only one copy per match
                End If
            Next sourceCell
        End If
    Next checkCell
End Sub


##Modul Macro - calc An [YY]

In [ ]:
Function FindFirstTwoDigitsOrCount(rng As Range, Optional matchValue As Variant) As Variant
    Dim cell As Range
    Dim firstTwoDigits As String
    Dim count As Integer
    Dim result As Variant

    count = 0

    For Each cell In rng
        If Len(cell.Value) >= 2 And IsNumeric(Left(cell.Value, 2)) Then
            firstTwoDigits = Left(cell.Value, 2)
            If IsMissing(matchValue) Then
                ' If matchValue is not provided, return the first two digits found.
                FindFirstTwoDigitsOrCount = firstTwoDigits
                Exit Function
            ElseIf matchValue = Val(firstTwoDigits) Then
                ' If matchValue is provided and matches the first two digits, increment the count.
                count = count + 1
            End If
        End If
    Next cell

    If IsMissing(matchValue) Then
        ' If no match was found and no matchValue was provided, return 0 or some indicator.
        FindFirstTwoDigitsOrCount = "No match found"
    Else
        ' If a matchValue was provided, return the count.
        FindFirstTwoDigitsOrCount = count
    End If
End Function


## Modul Macro - Verificare "V" / Extern

In [ ]:
Function CheckAndCountVCells(rng As Range) As String
    Dim cell As Range
    Dim count As Integer
    Dim vExists As Boolean

    count = 0
    vExists = False ' Assume "V" does not exist initially

    For Each cell In rng
        If InStr(1, cell.Value, "V") > 0 Then
            count = count + 1
            vExists = True
        End If
    Next cell

    ' Decide what to return based on whether "V" was found and the count
    If count > 0 Then
        ' If "V" exists in any cell, you can return "Extern" or the count of such cells
        ' To return "Extern" and the count: CheckAndCountVCells = "Extern, Count: " & count
        ' To return just "Extern":
        CheckAndCountVCells = "Extern"
    Else
        CheckAndCountVCells = "Intern"
    End If
End Function
